# Problem Statement

Fradulent Medicare claims is a serious problem impacting the healthcare industry. The purpose of this project is to build a predictive model to detect such claims and prevent insurance companies from making payments for those claims.

# Project Structure

* Data Understading
* Data Prepration
* Exploratory Data Analysis & Feature Engineering
* Modelling
* Model Evaluation
* Deployment

# Data Understanding

# Data Preparation

# Exploratory Data Analysis

# Modeling